In [45]:
### import libraries and data; define fncs
import jax
import jax.numpy as jnp
from jax import jit
import jax.random as rnd
import numpy as np
import matplotlib
matplotlib.use('Qt5Agg')  # Use TkAgg backend to prevent segmentation fault
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import functools
# import matplotlib.ticker as ticker

def mod_(x):
    return (x + np.pi) % (2 * np.pi) - np.pi

def load_data(str_):
    path_ = os.path.dirname(os.getcwd()) + '/sc_project/test_data/' # Path(__file__).resolve().parents[1]
    with open(path_+str_,'rb') as file_:
        # param = pickle.load(file_)
        param_ = jnp.load(file_,allow_pickle=True)
    return param_

file_ = 'move_loop_v1_26_07-093804.pkl' # move_loop_v1__25_07-161050.pkl
(r_arr,rt_arr,sample_arr,pos_arr,dot_arr) = load_data(file_) # ,sel
print('r_arr.shape=',r_arr.shape,'rt_arr.shape=',rt_arr.shape,'sample_arr.shape=',sample_arr.shape,'pos_arr.shape=',pos_arr.shape,'dot_arr.shape=',dot_arr.shape,'sel.shape=',sel.shape)

total_rewards = jnp.sum(r_arr, axis=1)
sorted_indices_desc = jnp.argsort(total_rewards)[::-1]
print('sorted_indices_desc=',sorted_indices_desc)

VMAPS = r_arr.shape[0]
IT = r_arr.shape[1]

r_arr.shape= (1500, 30) rt_arr.shape= (1500, 30) sample_arr.shape= (1500, 30) pos_arr.shape= (1500, 30, 2) dot_arr.shape= (1500, 30, 2) sel.shape= (1500, 1)
sorted_indices_desc= [1460  953  664 ...  104 1137  165]


In [47]:
# animation of timeseries data
# plt.ion()

# fncs
def mod_(x):
    return (x + np.pi) % (2 * np.pi) - np.pi

def animate(i, agent_t, agent, dot, dot_t, aperture, box_points): # ,agent,agent_x_series,agent_y_series,dot,dot_xy,x_box,y_box
    agent.set_data(mod_(agent_t[0,i]), mod_(agent_t[1,i]))
    dot.set_data(mod_(dot_t[0, i]), mod_(dot_t[1, i]))
    
    # Get box outline coordinates
    x_box, y_box = get_box_points(agent_t[:,i], aperture)
    
    # Apply mod_ to each of the coordinates
    x_box_mod = mod_(x_box)
    y_box_mod = mod_(y_box)
    box_points.set_offsets(np.c_[x_box_mod, y_box_mod])

    if i == 0:
        box_points.set_color('green')
    else:
        box_points.set_color('lightgrey')

    return agent, dot, box_points,

def get_box_points(agent_t, aperture, density=100):
    
    # Horizontal lines (top and bottom of box)
    x_horizontal = np.linspace(agent_t[0] - aperture, agent_t[0] + aperture, density)
    y_top = np.full(density, agent_t[1] + aperture)
    y_bottom = np.full(density, agent_t[1] - aperture)
    
    # Vertical lines (left and right of box)
    y_vertical = np.linspace(agent_t[1] - aperture, agent_t[1] + aperture, density)
    x_left = np.full(density, agent_t[0] - aperture)
    x_right = np.full(density, agent_t[0] + aperture)
    
    # Combine to create the box outline
    x_combined = np.concatenate([x_left, x_right, x_horizontal, x_horizontal])
    y_combined = np.concatenate([y_vertical, y_vertical, y_bottom, y_top])
    
    return x_combined, y_combined

# generate timeseries, hp's
time_points = np.linspace(0,IT-1,IT)
aperture = np.pi/2
interval = 300 # ms

# select data
k = 1460 # 3 # 5
agent_t = pos_arr[k,:,:].T # IT,VMAP
dot_t = dot_arr[k,:,:].T # IT,VMAP

# fig/ticks/labels
fig, ax = plt.subplots(figsize=(6, 6))  # square figure
ax.set_xlim(-np.pi, np.pi)
ax.set_ylim(-np.pi, np.pi)
ax.set_aspect('equal', 'box')  # Ensuring the aspect ratio is equal
ticks = np.arange(-np.pi, np.pi+1, np.pi/2)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
labels = ["$-\pi$", "$-\pi/2$", "0", "$\pi/2$", "$\pi$"]
ax.set_xticklabels(labels)
ax.set_yticklabels(labels)
ax.legend(loc='lower center', bbox_to_anchor=(0.5, 1.05), ncol=3)

# initialise agent/dot/box
agent, = ax.plot(mod_(agent_t[0,0]), mod_(agent_t[1,0]), 'k+', markersize=12, label='Agent')
dot, = ax.plot(mod_(dot_t[0,0]), mod_(dot_t[1,0]), 'rx', markersize=8, label='Dot')
box_points = ax.scatter([], [], color='lightgrey', s=1)

# animate
animate_partial = functools.partial(
    animate, 
    agent_t=agent_t,
    agent=agent,
    dot=dot,
    dot_t=dot_t,
    aperture=aperture,
    box_points=box_points
)

ani = animation.FuncAnimation(
    fig, animate_partial, frames=len(time_points), interval=interval, repeat=True
)

ax.legend()
plt.show()

print('dot_t=',dot_t.shape,dot_t,'agent_t=',agent_t.shape,agent_t)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


dot_t= (2, 30) [[  0.07954254   0.19552617   0.3115098    0.42749342   0.54347706
    0.65946066   0.77544427   0.8914279    1.0074115    1.1233951
    1.2393787    1.3553623    1.4713459    1.5873295    1.7033131
    1.8192967    1.9352803    2.051264     2.1672478    2.2832315
    2.3992152    2.515199     2.6311827    2.7471664    2.8631501
    2.9791338    3.0951176    3.2111013    3.327085     3.4430687 ]
 [ -1.0417987   -1.5177653   -1.9937319   -2.4696984   -2.945665
   -3.4216313   -3.8975978   -4.3735642   -4.8495307   -5.325497
   -5.8014636   -6.27743     -6.7533965   -7.229363    -7.7053294
   -8.181296    -8.657263    -9.133229    -9.609196   -10.085162
  -10.561129   -11.037095   -11.513062   -11.989028   -12.464994
  -12.940961   -13.416927   -13.892894   -14.36886    -14.844827  ]] agent_t= (2, 30) [[  0.47144282   0.68088245   0.8903221    1.0997617    1.3092014
    1.518641     1.7280806    1.9375203    1.0997622   -0.3663144
    1.0997622    1.3092018    1.5186415   

/var/folders/hp/sdqbmwnd5m14lcp1k2qjnkx00000gn/T/ipykernel_15413/1004529139.py:9: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  agent.set_data(mod_(agent_t[0,i]), mod_(agent_t[1,i]))
/var/folders/hp/sdqbmwnd5m14lcp1k2qjnkx00000gn/T/ipykernel_15413/1004529139.py:10: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  dot.set_data(mod_(dot_t[0, i]), mod_(dot_t[1, i]))


In [ ]:
# example animation

# generate timeseries
time_points = np.linspace(0, 10, 100)
agent_x = np.sin(time_points) * np.pi  
agent_y = np.cos(time_points) * np.pi

# random initial position / velocity
dot_speed = 10  # Speed of the dot
initial_angle = 2 * np.pi * np.random.rand((2)) - np.pi  # Random value between [-π, π]
angular_velocity = (2 * np.pi * np.random.rand((2)) - np.pi) / 10  # Random angular velocity

# angular trajectory of dot
dot_xy = mod_(initial_angle + dot_speed*np.outer(angular_velocity, time_points).T).T

# fig/ticks/labels
fig, ax = plt.subplots(figsize=(6, 6))  # square figure
ax.set_xlim(-np.pi, np.pi)
ax.set_ylim(-np.pi, np.pi)
ax.set_aspect('equal', 'box')  # Ensuring the aspect ratio is equal
ticks = np.arange(-np.pi, np.pi+1, np.pi/2)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
labels = ["$-\pi$", "$-\pi/2$", "0", "$\pi/2$", "$\pi$"]
ax.set_xticklabels(labels)
ax.set_yticklabels(labels)

# initialise agent/dot
agent, = ax.plot(agent_x[0], agent_y[0], 'k+', markersize=12, label='Agent')
dot, = ax.plot(dot_xy[0,0], dot_xy[1,0], 'rx', markersize=8, label='Dot')
box_size = np.pi
# agent_box1 = plt.Rectangle((agent_x[0]-box_size/2, agent_y[0]-box_size/2), box_size, box_size, fill=False, color='lightgrey')
# agent_box2 = plt.Rectangle((-3*np.pi, -3*np.pi), box_size, box_size, fill=False, color='lightgrey') # Initially out of view
# ax.add_patch(agent_box1)
# ax.add_patch(agent_box2)
# box_x = [agent_x[0]-box_size/2, agent_x[0]+box_size/2, agent_x[0]+box_size/2, agent_x[0]-box_size/2, agent_x[0]-box_size/2]
# box_y = [agent_y[0]-box_size/2, agent_y[0]-box_size/2, agent_y[0]+box_size/2, agent_y[0]+box_size/2, agent_y[0]-box_size/2]
# agent_box_line, = ax.plot(box_x, box_y, color='lightgrey')
line, = ax.plot([], [], color='lightgrey')

def get_box_outline(agent_x, agent_y, box_size):
    # Define the box corners
    corners_x = [agent_x-box_size/2, agent_x+box_size/2, agent_x+box_size/2, agent_x-box_size/2, agent_x-box_size/2]
    corners_y = [agent_y-box_size/2, agent_y-box_size/2, agent_y+box_size/2, agent_y+box_size/2, agent_y-box_size/2]
    return corners_x, corners_y

def animate(i):
    agent_x = agent_x[i]
    agent_y = agent_y[i]
    
    # Get box outline coordinates
    corners_x, corners_y = get_box_outline(agent_x, agent_y, box_size)

    # Apply mod_ to box outline coordinates
    corners_x_mod = [mod_(x) for x in corners_x]
    corners_y_mod = [mod_(y) for y in corners_y]
    
    line.set_data(corners_x_mod, corners_y_mod)

    return line,

# animate
# def animate(i):
#     agent.set_data(agent_x[i], agent_y[i])
#     dot.set_data(dot_xy[0, i], dot_xy[1, i])

#     # Calculate the 4 corner coordinates of the main box
#     top_left = [mod_(agent_x[i] - box_size/2), mod_(agent_y[i] + box_size/2)]
#     top_right = [mod_(agent_x[i] + box_size/2), mod_(agent_y[i] + box_size/2)]
#     bottom_left = [mod_(agent_x[i] - box_size/2), mod_(agent_y[i] - box_size/2)]
#     bottom_right = [mod_(agent_x[i] + box_size/2), mod_(agent_y[i] - box_size/2)]

#     # Determine box spans based on wrapped coordinates
#     spans_right = top_right[0] < top_left[0]
#     spans_left = bottom_left[0] > bottom_right[0]
#     spans_top = top_left[1] < bottom_left[1]
#     spans_bottom = bottom_left[1] > top_left[1]

#     # Adjust the main agent box to always be in view
#     agent_box1.set_xy((agent_x[i] - box_size/2, agent_y[i] - box_size/2))
#     agent_box1.set_width(box_size if not spans_right else np.pi + agent_x[i])
#     agent_box1.set_height(box_size if not spans_top else np.pi + agent_y[i])

#     # Adjust the secondary box for the right side
#     if spans_right:
#         agent_box2.set_xy((-np.pi, agent_y[i] - box_size/2))
#         agent_box2.set_width(top_right[0] + np.pi)
#         agent_box2.set_height(box_size)

#     # Adjust for the left side
#     elif spans_left:
#         agent_box2.set_xy((agent_x[i] + box_size/2, agent_y[i] - box_size/2))
#         agent_box2.set_width(-bottom_left[0] + np.pi)
#         agent_box2.set_height(box_size)

#     # Adjust for the top side
#     elif spans_top:
#         agent_box2.set_xy((agent_x[i] - box_size/2, -np.pi))
#         agent_box2.set_width(box_size)
#         agent_box2.set_height(top_left[1] + np.pi)

#     # Adjust for the bottom side
#     elif spans_bottom:
#         agent_box2.set_xy((agent_x[i] - box_size/2, agent_y[i] + box_size/2))
#         agent_box2.set_width(box_size)
#         agent_box2.set_height(-bottom_left[1] + np.pi)

#     else:
#         # If no wrapping, hide the secondary box
#         agent_box2.set_xy((-3*np.pi, -3*np.pi))
#         agent_box2.set_width(0)
#         agent_box2.set_height(0)

#     return agent, dot, agent_box1, agent_box2,
# def animate(i):
#     agent.set_data(agent_x[i], agent_y[i])
#     dot.set_data(dot_xy[0,i], dot_xy[1,i])
    
#     # Update box lines
#     box_x = [agent_x[i]-box_size/2, agent_x[i]+box_size/2, agent_x[i]+box_size/2, agent_x[i]-box_size/2, agent_x[i]-box_size/2]
#     box_y = [agent_y[i]-box_size/2, agent_y[i]-box_size/2, agent_y[i]+box_size/2, agent_y[i]+box_size/2, agent_y[i]-box_size/2]
#     agent_box_line.set_xdata(mod_(np.array(box_x)))
#     agent_box_line.set_ydata(mod_(np.array(box_y)))
    
#     return agent, dot, agent_box_line,

ani = animation.FuncAnimation(
    fig, animate, frames=len(time_points), interval=50, repeat=True
)

ax.legend()
plt.show()
# plt.ion()

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
matplotlib.use('Qt5Agg')
import functools
# plt.ion()

def mod_(x):
    return (x + np.pi) % (2 * np.pi) - np.pi

def animate(i, agent_t, agent, dot, dot_t, box_size, box_points): # ,agent,agent_x_series,agent_y_series,dot,dot_xy,x_box,y_box
    agent.set_data(agent_t[0,i], agent_t[1,i])
    dot.set_data(dot_t[0, i], dot_t[1, i])
    
    # Get box outline coordinates
    x_box, y_box = get_box_points(agent_t[:,i], box_size)
    
    # Apply mod_ to each of the coordinates
    x_box_mod = mod_(x_box)
    y_box_mod = mod_(y_box)
    box_points.set_offsets(np.c_[x_box_mod, y_box_mod])

    return agent, dot, box_points,

def get_box_points(agent_t, box_size, density=100):
    half_size = box_size / 2
    
    # Horizontal lines (top and bottom of box)
    x_horizontal = np.linspace(agent_t[0] - half_size, agent_t[0] + half_size, density)
    y_top = np.full(density, agent_t[1] + half_size)
    y_bottom = np.full(density, agent_t[1] - half_size)
    
    # Vertical lines (left and right of box)
    y_vertical = np.linspace(agent_t[1] - half_size, agent_t[1] + half_size, density)
    x_left = np.full(density, agent_t[0] - half_size)
    x_right = np.full(density, agent_t[0] + half_size)
    
    # Combine to create the box outline
    x_combined = np.concatenate([x_left, x_right, x_horizontal, x_horizontal])
    y_combined = np.concatenate([y_vertical, y_vertical, y_bottom, y_top])
    
    return x_combined, y_combined

# generate timeseries, hp's
time_points = np.linspace(0, 10, 100)
box_size = np.pi

# fig/ticks/labels
fig, ax = plt.subplots(figsize=(6, 6))  # square figure
ax.set_xlim(-np.pi, np.pi)
ax.set_ylim(-np.pi, np.pi)
ax.set_aspect('equal', 'box')  # Ensuring the aspect ratio is equal
ticks = np.arange(-np.pi, np.pi+1, np.pi/2)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
labels = ["$-\pi$", "$-\pi/2$", "0", "$\pi/2$", "$\pi$"]
ax.set_xticklabels(labels)
ax.set_yticklabels(labels)

# gen agent data
agent_x_series = np.sin(time_points) * np.pi  
agent_y_series = np.cos(time_points) * np.pi
agent_t = np.vstack([agent_x_series, agent_y_series])
box_points = ax.scatter([], [], color='lightgrey', s=1)  # s is the marker size
agent, = ax.plot(agent_t[0,0], agent_t[1,0], 'k+', markersize=12, label='Agent')

# gen dot data
dot_speed = 10  # Speed of the dot
initial_angle = 2 * np.pi * np.random.rand((2)) - np.pi  # Random value between [-π, π]
angular_velocity = (2 * np.pi * np.random.rand((2)) - np.pi) / 10  # Random angular velocity
dot_t = mod_(initial_angle + dot_speed*np.outer(time_points, angular_velocity)).T
dot, = ax.plot(dot_t[0,0], dot_t[1,0], 'rx', markersize=8, label='Dot')

# animate
animate_partial = functools.partial(
    animate, 
    agent_t=agent_t, 
    agent=agent, 
    dot=dot, 
    dot_t=dot_t, 
    box_size=box_size, 
    box_points=box_points
)

ani = animation.FuncAnimation(
    fig, animate_partial, frames=len(time_points), interval=50, repeat=True
)

ax.legend()
plt.show()

/var/folders/hp/sdqbmwnd5m14lcp1k2qjnkx00000gn/T/ipykernel_15413/1258831286.py:12: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  agent.set_data(agent_t[0,i], agent_t[1,i])
/var/folders/hp/sdqbmwnd5m14lcp1k2qjnkx00000gn/T/ipykernel_15413/1258831286.py:13: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  dot.set_data(dot_t[0, i], dot_t[1, i])
